In [2]:
## importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#from help_fun import * # style not good - better: import explicitely
import numpy as np
import cv2
import math
%matplotlib inline

In [3]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)


In [ ]:
## read image:
image = mpimg.imread('test_images/solidWhiteRight.jpg')
print('shape of RGB image:\t ', image.shape)

# find maximum x and y values
y_max, x_max, _ = image.shape

# plot rgb-image:
plt.figure()
plt.imshow(image)
plt.title('\'image\' - solidWhiteRight image (RGB)')
#plt.show

In [ ]:
## convert image into grayscale:
img_gray = grayscale(image)
print('shape of grayscale image:', img_gray.shape)

# plot grayscale image:
plt.figure()
plt.imshow(img_gray, cmap='gray')
plt.title('\'img_gray\' - grayscale image')
#plt.show

In [ ]:
## blur image:
gray_blr = gaussian_blur(img_gray, 5)
plt.figure()
plt.imshow(gray_blr, cmap='gray')
plt.title('\'gray_blr\' - blurred grayscale image')

## apply canny edge detector:
##### todo: write loop to compare (save) pictures with different threshold values (GUI?)
th_low = 100;
th_high = 200;
plt.figure()
canny_edge = canny(gray_blr, th_low, th_high)

plt.imshow(canny_edge, cmap='gray')
plt.title('\'canny_edge\' - image after canny edge detection')
#cv2.imwrite('test_images_output/canny_edge_100-200.jpg',canny_edge)

In [ ]:
## select region of interest: 
# (ROI was selected after canny edge detection because otherwise edges are detected at the boundaries of ROI)

# determine polygon vertices:
bottom_left = np.array([0, y_max])
bottom_right = np.array([x_max, y_max])
apex = np.array([round(x_max/2), round(y_max/2)])

# visualize ROI
image_roi = np.copy(image)
pts = np.array([[apex, bottom_left, bottom_left, bottom_right, bottom_right, apex]])
cv2.polylines(image_roi, pts, True, [255, 0, 0], 3)
plt.figure()
plt.imshow(image_roi)
plt.title('\'image_roi\' - Original image with ROI')


# Apply ROI on canny edge image
vertices = np.array([[bottom_left, bottom_right, apex]])
canny_roi = region_of_interest(canny_edge, vertices)
plt.figure()
plt.imshow(canny_roi, cmap='gray')
plt.title('\'canny_roi\' - Canny image with ROI')

In [ ]:
## hough transform:
rho = 1
theta = 1 * np.pi/180

### parameters to detect complete lines only with hough-transform: #threshold = 70 #min_line_len = 100 #max_line_gap = 140
threshold = 70
min_line_len = 50
max_line_gap = 50


# # outputs a new image with hough lines drawn on it
# line_img = hough_lines(canny_roi, rho, theta, threshold, min_line_len, max_line_gap)
# plt.figure()
# plt.imshow(line_img)
# plt.title('\'line_img\' - black image with hough lines drawn on it')


# Probabilistic Hough Line Transform (outputs the extremes of the detected lines)
lines = cv2.HoughLinesP(canny_roi, rho, theta, threshold, np.array([]), min_line_len, max_line_gap)
#print(lines)
# create copy of image
line_img2 = np.copy(image)
# draws lines on passed image
draw_lines(line_img2, lines, color=[255, 0, 0], thickness=2)
plt.figure()
plt.imshow(line_img2)
plt.title('\'line_img2\' - original image with hough lines drawn on it')

In [ ]:
def get_line_params(line):
    # function takes a single line (2 extreme points: x0(line[0,0]), y0(line[0,1]), x1(line[0,2]), y1(line[0,3]))
    # and returns the slope and offset as two separate variables (y=mx+b)
    ### how to get rid of the second dimension? (first zero)
    m = (line[0,3]-line[0,1])/(line[0,2]-line[0,0])
    b = ((line[0,2]*line[0,1]-line[0,0]*line[0,3])/(line[0,2]-line[0,0]))
    return m, b

# def get_slope(line):
#     m = (line[0,3]-line[0,1])/(line[0,2]-line[0,0])
#     return m

# def get_offset(line):
#     b = ((line[0,2]*line[0,1]-line[0,0]*line[0,3])/(line[0,2]-line[0,0]))
#     return b

In [ ]:
# separate if lines are left (slope<0) or right (slope>0) from car
# initialize parameteres for both left and right lanes
num_left = 0
m_left = 0
b_left = 0
num_right = 0
m_right = 0
b_right = 0

# initialize min and max values of lines:
min_y, min_x = image.shape[0:2]
max_x = max_y = 0
#print(image.shape)

for line in lines:
    m, b = get_line_params(line)
    if m < 0: # lines on the left
        num_left += 1 # increment left line counter
        m_left += m # add slope value to the others as part of averaging
        b_left += b # add offset value to the others as part of averaging
        #print('left m, b: ', np.round(m,decimals=3), np.round(b,decimals=3))
    elif m > 0: # lines on the right
        num_right += 1 # increment right line counter
        m_right += m # add slope value to the others
        b_right += b # add offset value to the others as part of averaging
        #print('right m, b: ', np.round(m,decimals=3), np.round(b,decimals=3))
    else:
        print('dont know what to do yet :(')
        ### don't know yet
# calculate average slopes and offsets:
if num_left > 0:
    m_mean_left = m_left/num_left
    b_mean_left = b_left/num_left
else:
    print('no left lines detected')
if num_right > 0:
    m_mean_right = m_right/num_right
    b_mean_right = b_right/num_right
else:
    print('no left lines detected')
    
#print('num_left, num_right: ', num_left, num_right)
#print('left mean m, b: ', m_mean_left, b_mean_left)
#print('right mean m, b: ', m_mean_right, b_mean_right)

In [ ]:
# def calc_intersection(m0, b0, m1, b1):
#     # returns the x and y coordinates of the intersection of two lines
#     x_is = (b1 - b0) / (m0 - m1)
#     y_is = m0 * (b1 - b0) / (m0 - m1) + b0
#     return int(x_is), int(y_is)

def calc_line_x_val(m, b, y):
    # returns the x value of a line at a given y value
    x = (y - b) / m
    return int(x)

In [ ]:
y_left_min = 350
x_left_min = calc_line_x_val(m_mean_left, b_mean_left, y_left_min)
x_left_max = calc_line_x_val(m_mean_left, b_mean_left, y_max)

y_right_min = 350
x_right_min = calc_line_x_val(m_mean_right, b_mean_right, y_right_min)
x_right_max = calc_line_x_val(m_mean_right, b_mean_right, y_max)

In [ ]:
#lines_final = np.array([[[x_left_max, y_max, x_intersect, y_intersect]],[[x_right_max, y_max, x_intersect, y_intersect]]])
lines_final = np.array([[[x_left_max, y_max, x_left_min, y_left_min]],[[x_right_max, y_max, x_right_min, y_right_min]]])
print(lines_final)

# create new copy from original image
image_cp = np.copy(image)

#draw_lines(image_cp, lines_final, [255, 0, 0], 10)

#image_black = np.zeros(image.shape) # not clear why not working
image_black = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
draw_lines(image_black, lines_final, [255, 0, 0], 13)

overlay = weighted_img(image_black, image_cp, 1, 1, 0)

plt.imshow(overlay)

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

In [ ]:
import os
os.listdir("test_images/")

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
## Parameters

# canny edge detection
th_low = 100
th_high = 200

# determine polygon vertices:
rows = canny_edge.shape[0]
cols = canny_edge.shape[1]
bottom_left = (0, rows)
bottom_right = (cols, rows)
### todo: optimize apex point
apex = (round(cols/2), round(rows/2))

# mask for roi
del_mask = np.array([[0, 0], bottom_left, apex, bottom_right, [cols, 0]])

# hough transform
rho = 1
theta = 1 * np.pi/180
### parameters to detect complete lines only with hough-transform: #threshold = 70 #min_line_len = 100 #max_line_gap = 140
threshold = 70 #70
min_line_len = 100 #50
max_line_gap = 140 #50

# define upper line extremes
y_left_min = y_right_min = 350

In [ ]:
#Functions

def average_left_right(lines):
    # separate if lines are left (slope<0) or right (slope>0) from car
    # initialize parameteres for both left and right lanes
    num_left = 0
    m_left = 0
    b_left = 0
    num_right = 0
    m_right = 0
    b_right = 0
    # initialize min and max values of lines:
    min_y, min_x = image.shape[0:2]
    max_x = max_y = 0
    for line in lines:
        m, b = get_line_params(line)
        if m < 0: # lines on the left
            num_left += 1 # increment left line counter
            m_left += m # add slope value to the others as part of averaging
            b_left += b # add offset value to the others as part of averaging
            #print('left m, b: ', np.round(m,decimals=3), np.round(b,decimals=3))
        elif m > 0: # lines on the right
            num_right += 1 # increment right line counter
            m_right += m # add slope value to the others
            b_right += b # add offset value to the others as part of averaging
            #print('right m, b: ', np.round(m,decimals=3), np.round(b,decimals=3))
        else:
            print('dont know what to do yet :(')
            ### don't know yet
    # calculate average slopes and offsets:
    if num_left > 0:
        m_avg_left = m_left/num_left
        b_avg_left = b_left/num_left
    else:
        print('no left lines detected')
        m_avg_left = np.inf
        b_avg_left = np.inf
    if num_right > 0:
        m_avg_right = m_right/num_right
        b_avg_right = b_right/num_right
    else:
        print('no left lines detected')
        m_avg_right = np.inf
        b_avt_right = np.inf
    return m_avg_left, b_avg_left, m_avg_right, b_avg_right

def calc_intersection(m0, b0, m1, b1):
    # returns the x and y coordinates of the intersection of two lines
    x_is = (b1 - b0) / (m0 - m1)
    y_is = m0 * (b1 - b0) / (m0 - m1) + b0
    return int(x_is), int(y_is)

def calc_line_x_val(m, b, y):
    # returns the x value of a line at a given y value
    x = (y - b) / m
    return int(x)

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    # convert into grayscale image
    img_gray = grayscale(image)
    
    # blur image
    gray_blr = gaussian_blur(img_gray, 5)
    
    # apply canny edge detector
    canny_edge = canny(gray_blr, th_low, th_high)
    
    # (create another copy of the image and) mask everything exept the region of interest out:
    #canny_roi = np.copy(canny_edge)
    cv2.fillPoly(canny_edge, [del_mask], (0, 0, 0))
    
    # Probabilistic Hough Line Transform (outputs the extremes of the detected lines)
    lines = cv2.HoughLinesP(canny_edge, rho, theta, threshold, np.array([]), min_line_len, max_line_gap)
    m_avg_left, b_avg_left, m_avg_right, b_avg_right = average_left_right(lines)
    
    line_img2 = np.copy(image)
    # draws lines on passed image
    draw_lines(line_img2, lines, color=[255, 0, 0], thickness=2)
    
    return line_img2
    
    
    
    # calculate intersection point
    x_intersect, y_intersect = calc_intersection(m_avg_left, b_avg_left, m_avg_right, b_avg_right)
    
    # calculate line extremes
    x_left_min = calc_line_x_val(m_avg_left, b_avg_left, y_left_min)
    x_left_max = calc_line_x_val(m_avg_left, b_avg_left, y_max)
    x_right_min = calc_line_x_val(m_avg_right, b_avg_right, y_right_min)
    x_right_max = calc_line_x_val(m_avg_right, b_avg_right, y_max)
    
    # calculate final lines (averaged and extrapolated)
    print(type(x_left_max), type(y_max), type(x_left_min), type(y_left_min), type(x_right_max), type(y_max), type(x_right_min), type(y_right_min))
    lines_final = np.array([[[x_left_max, y_max, x_left_min, y_left_min]],[[x_right_max, y_max, x_right_min, y_right_min]]])
    
    image_black = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    draw_lines(image_black, lines_final, [255, 0, 0], 13)

    # create new copy from original image
    image_copy = np.copy(image)
    #overlay = weighted_img(image_black, image_cp, 1, 1, 0)
    image_final = weighted_img(image_black, image_copy, 1, 1, 0)
    
    return image_final

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

# backup:

In [ ]:
#### overlay:

# enlarge figure:
#fig = plt.figure()
#size = fig.get_size_inches()
#fig = plt.figure(figsize=size*1)